<a href="https://colab.research.google.com/github/verevar/big-data-challenge/blob/main/Grocery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.1.3'
# spark_version = 'spark-3.<version number>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-09-12 22:56:34--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.81MB/s    in 0.2s    

2022-09-12 22:56:35 (4.81 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Groceries").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Grocery_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

In [ ]:
 grocery_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Grocery_v1_00.tsv.gz"), sep="\t", header=True)

#Show DataFrame
grocery_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42521656|R26MV8D0KG6QI6|B000SAQCWC|     159713740|The Cravings Plac...|         Grocery|          5|            0|          0|   N|                Y|Using these for y...|As a family aller...| 2015-08-31|
|         US|   12049833|R1OF8GP57AQ1A0|B00509LVIQ|     138680402|Mauna Loa Macadam...|         Grocery|          5|    

In [ ]:
grocery_df = grocery_df.dropna()

In [ ]:
grocery_df = grocery_df.dropDuplicates()

In [ ]:
#Count the number of records (rows) in the dataset.
grocery_df.count()

2402212

In [ ]:
# Be sure that the DataFrames match in data type and in column name.
from pyspark.sql.types import * 
grocery_df = grocery_df.withColumn("customer_id", grocery_df["customer_id"].cast(IntegerType()))\
                          .withColumn("product_parent", grocery_df["product_parent"].cast(IntegerType()))\
                          .withColumn("star_rating", grocery_df["star_rating"].cast(IntegerType()))\
                          .withColumn("helpful_votes", grocery_df["helpful_votes"].cast(IntegerType()))\
                          .withColumn("total_votes", grocery_df["total_votes"].cast(IntegerType()))\
                          .withColumn("review_date", grocery_df["review_date"].cast(DateType()))

In [ ]:
grocery_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
review_id = grocery_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id = review_id.dropDuplicates()
review_id = review_id.dropna()
review_id.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R11C0U5T9MODT2|   29404388|B001EO7LVM|     698009410| 2015-01-15|
|R131A73RHERLA1|   38349983|B001EQ5F9K|     802359601| 2011-06-29|
|R14MILT1NQE7SD|   46987164|B0030F3LCQ|     169901314| 2015-06-11|
|R1549CY2AKKRW0|   14427547|B001J3ZUQ6|     229167817| 2011-06-10|
|R15ZYDVLQ4C65Y|   31810294|B00AO7HZWA|     493497653| 2014-11-30|
|R17N83P9MCLENF|   37162088|B00061KXV4|      53038075| 2013-06-09|
|R18LR4S8LVPFI0|   45812419|B000LKYMOO|     935764386| 2007-06-09|
|R18SXC6ZP5KZ3H|   16407583|B001688E1C|     193521707| 2014-12-20|
|R19O10L0UM794T|   26117265|B001HTKV1G|     333105984| 2013-07-12|
|R1BQFX2VD64NKR|    9465074|B0009TMU0K|      61518983| 2015-03-10|
|R1CDN8UXYTYWIA|   15503046|B000P4ZUB0|     756228435| 2012-02-08|
|R1FTO5ZW74R2UK|   26748513|B0048KIQ90|     193554714| 2014-11

In [ ]:
review_id.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
products_df = grocery_df.select(["product_id","product_title"])
products_df = products_df.dropDuplicates()
products_df = products_df.dropna()

products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B000N308RE|Tillamook Vintage...|
|B0052UCCI6|Green Mountain Br...|
|B006TU9EQ6|Raspberry Gourmet...|
|B00CWTZCI2|Torani Sugar Free...|
|B0005ZZ53C|Welch's Grape Sod...|
|B00KSF4LZM|Angelino's Coffee...|
|B009IQLTFW|Zevia All Natural...|
|B0000SWZX2|Black Mountain Go...|
|B0012X9CIK|Kettle Brand Krin...|
|B000LL0R8I|Vita Coco Coconut...|
|B00JBBX5UQ|Buffalo Bills Lol...|
|B004N5FR06|Giant Reese’s Mil...|
|B006TH3XXY|Vero Spicy Mango ...|
|B002QWQGUS|Back to Nature Ma...|
|B00GDGLDWE|trücup Low Acid C...|
|B000MWVCQW|Emerald Nuts Chip...|
|B005QQ1L10|White Lily All Pu...|
|B00J3U4Y5A|Raw Organic Cacao...|
|B00CVTFT2M|Kellogg's Special...|
|B00EHVN1S6|Bragg Nutritional...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
products_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [ ]:
customer_id = grocery_df.groupby("customer_id")\
              .agg({"customer_id": "count"})\
              .withColumnRenamed("count(customer_id)", "customer_count")
customer_id = customer_id.dropDuplicates()
customer_id = customer_id.dropna()

customer_id.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   19836511|             2|
|   37859085|             1|
|   48997737|            24|
|   42319435|            83|
|   19624806|             1|
|   52218500|             3|
|   50041220|             1|
|   44714015|            12|
|   27636670|             1|
|   24133679|             2|
|   35864889|            16|
|   12250773|             1|
|   16224491|             2|
|    2761659|             4|
|   49323498|             1|
|   13788774|             1|
|   25790304|             3|
|   28528294|            38|
|    2769839|             1|
|   14299971|             7|
+-----------+--------------+
only showing top 20 rows



In [ ]:
customer_id.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [ ]:
columns = ['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine']
vine_table = grocery_df[[columns]]
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R100KOICK3R3CF|          5|            0|          0|   N|
|R100PWDLZSNOJE|          5|            0|          0|   N|
|R10185C2H88C6O|          5|            0|          0|   N|
|R102A21SZ34VPM|          5|            0|          0|   N|
|R102A2XJDY3YPA|          1|            1|          1|   N|
|R102ASTZMWUDN5|          5|            2|          2|   N|
|R1038PDERZDPPA|          5|            0|          0|   N|
|R104C1VFOOUG35|          5|           14|         15|   N|
|R104HQ5X03UBJV|          5|            0|          0|   N|
|R1050WCB1ADCLZ|          1|           75|         80|   N|
|R1051UF4PE1RAM|          1|            0|          0|   N|
|R1056E5M8D9VZD|          5|            0|          0|   N|
|R105I0GFDTIXTV|          5|            2|          2|   N|
|R105P0QQJXKAXZ|          5|            

In [ ]:
vine_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)

